In [527]:
import pandas as pd
import re, os
import numpy as np
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

! pip install docxtpl
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Cm
import matplotlib.pyplot as plt
import datetime

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [528]:
df = pd.read_excel("/content/drive/My Drive/codigos_colab/planilha_coleta_30_11.xlsx")


1. Inserindo os casos da Ana Clara

In [529]:
df_1 = pd.read_excel('/content/drive/My Drive/codigos_colab/sorteio_0_v3_amostra_1_anaklink.xlsx')
df_2 =  pd.read_excel('/content/drive/My Drive/codigos_colab/sorteio_1_v3_amostra_1_Anaklink.xlsx')

df_1 = df_1.iloc[7:,:]
df_2 = df_2.iloc[:24,:]

df_es = pd.concat([df_1,df_2],ignore_index=True)

# df_es.drop_duplicates(inplace = True)

print(len(df_es))

df_filter = df['9 - Nome do tribunal que abrange o foro/comarca/subseção onde o processo foi distribuído'] == "TJES"
df_filtro_es = df[df_filter]

# print(df_filtro_es.iloc[:,1])

idx = [3150]

# todos os números do ES
df_filtro_es.drop(idx, inplace= True)

# print(len(df_filtro_es))

# print(df_es)

217


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [530]:
lista_num = df_es["Número do Processo"].to_list()

df_es_ajust = pd.DataFrame()

df_es_ajust["3 - Número do processo (se possível, colocar no padrão CNJ com pontos e traço. EX: 1000347-33.2021.8.26.0114):"] = lista_num

datetime_obj = datetime.datetime.strptime('11/2/2022 08:26:42', '%m/%d/%Y %H:%M:%S')
datetime_obj_2 = datetime.datetime.strptime('1/1/0001','%d/%m/%Y')
datetime_obj_3 = datetime.datetime.strptime('11/2/2022 08:27:42', '%m/%d/%Y %H:%M:%S')


df_es_ajust["Carimbo de data/hora"] = datetime_obj 
df_es_ajust["1 - Nome do pesquisadora"] = "Ana Clara"
df_es_ajust['3.1 - Caso descartado antecipadamente? (por motivos como "não encontrado no sistema", "número inválido", etc)'] = "Sim"
df_es_ajust['3.1.1. Em caso de resposta "sim" na pergunta anterior, informe o motivo:'] = "Não encontrado no sistema"
df_es_ajust['4 - Data da distribuição (colocar 010101 - para casos "sem informações")'] = datetime_obj_2
df_es_ajust["5 - É caso de interesse?"] = "não"
df_es_ajust["5.1 - Se a resposta anterior foi não, qual a razão da eliminação ?"] = "Processo não encontrado"
df_es_ajust["80 - fim do preenchimento"] = datetime_obj_3
df_es_ajust['Endereço de e-mail'] = "anaclaraklink@gmail.com"

print(len(df))
df = df.iloc[5:,:]
# print(df.iloc[0,1])
print(len(df))
df_final = pd.concat([df,df_es_ajust], ignore_index=True)

print(len(df_final))
# print(df_final)

4002
3997
4214


In [531]:
# limpando os números
numeros = df_final.iloc[:,3].to_list()

rgx_lp = re.compile("[^0-9]")

nm_limpos = []
for n in range(len(numeros)):
    # print(numeros[n])
    nm = re.sub(rgx_lp,"",numeros[n])
    # print(nm.strip())
    nm_limpos.append(nm.strip())

print(len(nm_limpos))

4214


In [532]:
# inserindo os Estados

dic_estados = {"801": "AC",
               "802": "AL",
               "803": "AP",
               "804": "AM",
               "805": "BA",
               "806": "CE",
               "807": "DF",
               "808": "ES",
               "809": "GO",
               "810": "MA",
               "811": "MT",
               "812": "MS",
               "813": "MG",
               "814": "PA",
               "815": "PB",
               "816": "PR",
               "817": "PE",
               "818": "PI",
               "819": "RJ",
               "820": "RN",
               "821": "RS",
               "822": "RO",
               "823": "RR",
               "824": "SC",
               "825": "SE",
               "826": "SP",
               "827": "TO",
               "501": "TRF1"
              }


# teste = "00021646120198060055"
#print(teste[13:16])

estados = []
a =0
for num in nm_limpos:
  trecho = num[13:16]
  try:
    estados.append(dic_estados[trecho])
  except:
    estados.append(None)
    a = a+1

# print(a)


df_final["Estado"] = estados


df_final["Estado"] = np.where(df_final["Estado"].isnull() == True, df_final["9 - Nome do tribunal que abrange o foro/comarca/subseção onde o processo foi distribuído"].str[2:],df_final["Estado"])

print(df_final["Estado"])

0       AC
1       AC
2       AC
3       AC
4       AC
        ..
4209    ES
4210    ES
4211    ES
4212    ES
4213    ES
Name: Estado, Length: 4214, dtype: object


In [533]:
print(len(df_final))
o = 0

4214


In [534]:
# 1 - AC

filter_ac = df_final["Estado"] == "AC"
df_filt_ac = df_final[filter_ac]

print(df_filt_ac.value_counts(subset = ['Endereço de e-mail']))

filter_2_ac = df_filt_ac['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_ac_1 = df_filt_ac[filter_2_ac]

filter_3_ac = df_filt_ac['Endereço de e-mail'] == "manuela.valenca@gmail.com"
df_filt_ac_2 = df_filt_ac[filter_3_ac]

df_filt_ac = pd.concat([df_filt_ac_1,df_filt_ac_2],ignore_index = True)
print(len(df_filt_ac))
# print(df_filt_ac)
o = o+len(df_filt_ac)
print(o)

Endereço de e-mail       
manuela.valenca@ufpe.br      132
anandaendo@usp.br              6
anandaendo@gmail.com           1
manuela.valenca@gmail.com      1
dtype: int64
133
133


In [535]:
# 2- AL

filter_al = df_final["Estado"] == "AL"
df_filt_al = df_final[filter_al]

print(df_filt_al.value_counts(subset = ['Endereço de e-mail']))


filter_2_al = df_filt_al['Endereço de e-mail'] == "marimzambom@gmail.com"
df_filt_al = df_filt_al[filter_2_al]

# filter_3 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com" # eliminar os 6 últimos da ANA Klink, possível bug
# df_filt_al_f = df_filt[filter_3]


# print(df_filt_al_f)
print(len(df_filt_al))
o = o+len(df_filt_al)
print(o)

Endereço de e-mail     
marimzambom@gmail.com      74
anaclaraklink@gmail.com     6
dtype: int64
74
207


In [536]:
# 3- AP

filter_ap = df_final["Estado"] == "AP"
df_filt_ap = df_final[filter_ap]

print(df_filt_ap.value_counts(subset = ['Endereço de e-mail']))


filter_2_ap = df_filt_ap['Endereço de e-mail'] == "marimzambom@gmail.com"
df_filt_ap_1 = df_filt_ap[filter_2_ap]

filter_3_ap = df_filt_ap['Endereço de e-mail'] == "marimzambom@hotmail.com"
df_filt_ap_2 = df_filt_ap[filter_3_ap]

df_filt_ap = pd.concat([df_filt_ap_1,df_filt_ap_2],ignore_index = True)

print(len(df_filt_ap))
o = o+len(df_filt_ap)
print(o)

Endereço de e-mail     
marimzambom@gmail.com      100
anaclaraklink@gmail.com      6
marimzambom@hotmail.com      1
dtype: int64
101
308


In [537]:
# 4- AM

filter_am = df_final["Estado"] == "AM"
df_filt_am = df_final[filter_am]

print(df_filt_am.value_counts(subset = ['Endereço de e-mail']))


filter_2_am = df_filt_am['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_am = df_filt_am[filter_2_am]

print(len(df_filt_am))
# print(df_filt_am)
o = o+len(df_filt_am)
print(o)

Endereço de e-mail     
manuela.valenca@ufpe.br    143
anaclaraklink@gmail.com      6
dtype: int64
143
451


In [538]:
# 5 - BA

filter_ba = df_final["Estado"] == "BA"
df_filt_ba = df_final[filter_ba]

print(df_filt_ba.value_counts(subset = ['Endereço de e-mail']))


filter_2_ba = df_filt_ba['Endereço de e-mail'] == "analiagalvaoalmeida12@gmail.com"
df_filt_ba = df_filt_ba[filter_2_ba]

print(len(df_filt_ba))
o = o+len(df_filt_ba)
print(o)

Endereço de e-mail             
analiagalvaoalmeida12@gmail.com    251
anaclaraklink@gmail.com              6
dtype: int64
251
702


In [539]:
# 6 - CE

filter_ce = df_final["Estado"] == "CE"
df_filt_ce = df_final[filter_ce]

print(df_filt_ce.value_counts(subset = ['Endereço de e-mail']))


filter_2_ce = df_filt_ce['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_ce = df_filt_ce[filter_2_ce]

print(len(df_filt_ce))
o = o+len(df_filt_ce)
print(o)

Endereço de e-mail     
manuela.valenca@ufpe.br    55
marimzambom@gmail.com       6
dtype: int64
55
757


In [540]:
# 7 - DF

filter = df_final["Estado"] == "DF"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))


filter_2 = df_filt['Endereço de e-mail'] == "anandaendo@usp.br"
df_filt_df = df_filt[filter_2]


# filter_3 = df_filt['Endereço de e-mail'] == "anandanendo@usp.br" ### e-mail errado enviou de novo!
# df_filt_df_2 = df_filt[filter_3]

print(len(df_filt_df))
o = o+len(df_filt_df)
print(o)

Endereço de e-mail         
anandaendo@usp.br              93
anabeatrizgpassos@gmail.com     7
gabriellenascimento@usp.br      7
anandanendo@usp.br              1
dtype: int64
93
850


In [541]:
# 8 -ES

filter = df_final["Estado"] == "ES"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_es = df_filt[filter_2]

print(len(df_filt_es))
o = o+len(df_filt_es)
print(o)

Endereço de e-mail     
anaclaraklink@gmail.com    271
manuela.valenca@ufpe.br      6
dtype: int64
271
1121


In [542]:
# 9 - GO

# não tem os de conferência

filter = df_final["Estado"] == "GO"
df_filt= df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "viviane.balbuglio@gmail.com"
df_filt_go = df_filt[filter_2]

print(len(df_filt_go))
o = o+len(df_filt_go)
print(o)

Endereço de e-mail         
viviane.balbuglio@gmail.com    93
manuela.valenca@ufpe.br         2
dtype: int64
93
1214


In [543]:
# 10 - MA    

# não tem os de conferência

filter = df_final["Estado"] == "MA"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "gabriellenascimento@usp.br "
df_filt_ma_1 = df_filt[filter_2]


filter_3 = df_filt['Endereço de e-mail'] == "gabriellenascimento@usp.br"
df_filt_ma_2 = df_filt[filter_3]


df_filt_ma = pd.concat([df_filt_ma_1,df_filt_ma_2],ignore_index = True)

print(len(df_filt_ma))
o = o+len(df_filt_ma)
print(o)

Endereço de e-mail            
gabriellenascimento@usp.br        293
manuela.valenca@ufpe.br             6
gabriellenascimento@usp.br          3
gabrellenascimento@usp.br           1
gabriellenasccimento@usp.br         1
gabriellenascimento@usp.org.br      1
gavbriellenascimento@usp.br         1
dtype: int64
296
1510


In [544]:
# 11 - MS

filter = df_final["Estado"] == "MS"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "viviane.balbuglio@gmail.com"
df_filt_ms_1 = df_filt[filter_2]


filter_3 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_ms_2 = df_filt[filter_3]


df_filt_ms = pd.concat([df_filt_ms_1,df_filt_ms_2],ignore_index = True)

print(len(df_filt_ms))
o = o+len(df_filt_ms)
print(o)

Endereço de e-mail         
viviane.balbuglio@gmail.com    73
anaclaraklink@gmail.com        10
marimzambom@gmail.com           6
dtype: int64
83
1593


In [545]:
# 12 - MG

filter = df_final["Estado"] == "MG"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_mg = df_filt[filter_2]

print(len(df_filt_mg))
o = o+len(df_filt_mg)
print(o)

Endereço de e-mail         
manuela.valenca@ufpe.br        60
anabeatrizgpassos@gmail.com     6
dtype: int64
60
1653


In [546]:
# 13 - MT

filter = df_final["Estado"] == "MT"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_mt = df_filt[filter_2]

## endereços da ana clara errados

print(len(df_filt_mt))
o = o+len(df_filt_mt)
print(o)

Endereço de e-mail         
anaclaraklink@gmail.com        263
viviane.balbuglio@gmail.com      6
anaclaraklinK@gmail.com          1
anaclarklink@gmail.com           1
dtype: int64
263
1916


In [547]:
# 14 - PA


filter = df_final["Estado"] == "PA"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "marimzambom@gmail.com"
df_filt_pa = df_filt[filter_2]



## endereços da Mari errados

# filter_3 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
# df_filt_pa_b = df_filt[filter_3]


# print(df_filt_pa_b)

print(len(df_filt_pa))
o = o+len(df_filt_pa)
print(o)

Endereço de e-mail     
marimzambom@gmail.com      234
anaclaraklink@gmail.com      6
marimzambom@gmai.com         1
dtype: int64
234
2150


In [548]:
# 15 - PB

filter = df_final["Estado"] == "PB"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "marimzambom@gmail.com"
df_filt_pb = df_filt[filter_2]


print(len(df_filt_pb))
o = o+len(df_filt_pb)
print(o)

Endereço de e-mail   
marimzambom@gmail.com    104
anandaendo@usp.br          6
dtype: int64
104
2254


In [549]:
# 16 - PE


filter = df_final["Estado"] == "PE"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anandaendo@usp.br"
df_filt_pe = df_filt[filter_2]


print(len(df_filt_pe))
o = o+len(df_filt_pe)
print(o)

Endereço de e-mail     
anandaendo@usp.br          312
manuela.valenca@ufpe.br      5
dtype: int64
312
2566


In [550]:
# 17 - PI

filter = df_final["Estado"] == "PI"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_pi = df_filt[filter_2]


print(len(df_filt_pi))
o = o+len(df_filt_pi)
print(o)

Endereço de e-mail         
manuela.valenca@ufpe.br        64
viviane.balbuglio@gmail.com     6
dtype: int64
64
2630


In [551]:
# 18 - PR


filter = df_final["Estado"] == "PR"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "viviane.balbuglio@gmail.com"
df_filt_pr = df_filt[filter_2]


print(len(df_filt_pr))
o = o+len(df_filt_pr)
print(o)

Endereço de e-mail         
viviane.balbuglio@gmail.com    135
marimzambom@gmail.com            6
dtype: int64
135
2765


In [552]:
# 19 - RJ


filter = df_final["Estado"] == "RJ"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "manuela.valenca@ufpe.br"
df_filt_rj = df_filt[filter_2]


print(len(df_filt_rj))
o = o+len(df_filt_rj)
print(o)

Endereço de e-mail     
manuela.valenca@ufpe.br    87
marimzambom@gmail.com       6
dtype: int64
87
2852


In [553]:
# 20 - RN


filter = df_final["Estado"] == "RN"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "marimzambom@gmail.com"
df_filt_rn = df_filt[filter_2]


print(len(df_filt_rn))
o = o+len(df_filt_rn)
print(o)

Endereço de e-mail       
marimzambom@gmail.com        144
mairarmachado@hotmail.com      6
anaclaraklink@gmail.com        1
dtype: int64
144
2996


In [554]:
# 21 - RO


filter = df_final["Estado"] == "RO"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_ro = df_filt[filter_2]


print(len(df_filt_ro))
o = o+len(df_filt_ro)
print(o)

Endereço de e-mail             
anaclaraklink@gmail.com            130
analiagalvaoalmeida12@gmail.com      6
anaclaraklink@gmail.ccom             1
anaclaraklink@gmail.vom              1
dtype: int64
130
3126


In [555]:
# 22 - RR


filter = df_final["Estado"] == "RR"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anandaendo@usp.br"
df_filt_rr = df_filt[filter_2]


print(len(df_filt_rr))
o = o+len(df_filt_rr)
print(o)

Endereço de e-mail         
anandaendo@usp.br              199
viviane.balbuglio@gmail.com      6
ananda@usp.br                    1
anandaendi@usp.br                1
dtype: int64
199
3325


In [556]:
# 23 - RS


filter = df_final["Estado"] == "RS"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anandaendo@usp.br"
df_filt_rs = df_filt[filter_2]


print(len(df_filt_rs))
o = o+len(df_filt_rs)
print(o)

Endereço de e-mail     
anandaendo@usp.br          221
manuela.valenca@ufpe.br      6
anandaendo@usp.com           1
dtype: int64
221
3546


In [557]:
# 24 - SC

filter = df_final["Estado"] == "SC"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "mairarmachado@hotmail.com"
df_filt_sc = df_filt[filter_2]


print(len(df_filt_sc))
o = o+len(df_filt_sc)
print(o)

Endereço de e-mail       
mairarmachado@hotmail.com    164
marimzambom@gmail.com          6
dtype: int64
164
3710


In [558]:
# 25 - SE

filter = df_final["Estado"] == "SE"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_se = df_filt[filter_2]


print(len(df_filt_se))
o = o+len(df_filt_se)
print(o)

Endereço de e-mail             
anaclaraklink@gmail.com            83
anandaendo@usp.br                   6
analiagalvaoalmeida12@gmail.com     1
dtype: int64
83
3793


In [559]:
# 26 - SP

filter = df_final["Estado"] == "SP"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "viviane.balbuglio@gmail.com"
df_filt_sp_1 = df_filt[filter_2]

filter_3 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_sp_2 = df_filt[filter_3]


df_filt_sp = pd.concat([df_filt_sp_1,df_filt_sp_2],ignore_index = True)

print(len(df_filt_sp))
o = o+len(df_filt_sp)
print(o)

Endereço de e-mail         
viviane.balbuglio@gmail.com    59
anaclaraklink@gmail.com        29
manuela.valenca@ufpe.br         6
anandaendo@usp.br               1
dtype: int64
88
3881


In [560]:
# 27 - TO

filter = df_final["Estado"] == "TO"
df_filt = df_final[filter]

print(df_filt.value_counts(subset = ['Endereço de e-mail']))

filter_2 = df_filt['Endereço de e-mail'] == "anaclaraklink@gmail.com"
df_filt_to = df_filt[filter_2]


print(len(df_filt_to))
o = o+len(df_filt_to)
print(o)

Endereço de e-mail         
anaclaraklink@gmail.com        130
viviane.balbuglio@gmail.com      6
anaclaraklink@gmail.con          2
dtype: int64
130
4011


In [561]:
print(o)

4011


In [562]:
df_final_aj = pd.concat([df_filt_ac,df_filt_al,df_filt_ap,df_filt_am,df_filt_ba,df_filt_ce,df_filt_df,df_filt_es,df_filt_go,df_filt_ma,df_filt_ms,df_filt_mg,df_filt_mt,df_filt_pa,df_filt_pb,df_filt_pe,df_filt_pi,df_filt_pr,df_filt_rj,df_filt_rn,df_filt_ro,df_filt_rr,df_filt_rs,df_filt_sc,df_filt_se,df_filt_sp,df_filt_to],ignore_index = True)
print(len(df_final_aj))

4011


2 - Fazendo a inserção das respostas dos estagiários

In [563]:
tabela_2 = pd.read_excel("/content/drive/My Drive/codigos_colab/fomularios_estag.xlsx")
print(len(tabela_2))
# print(tabela_2)
tabela_2 = tabela_2.iloc[1:,:]
print(len(tabela_2))

158
157


In [564]:
tabela_2["numero_limpo"] = tabela_2.iloc[:,3].replace('[^0-9]',value="", regex = True)
df_final_aj["numero_limpo"] = df_final_aj.iloc[:,3].replace('[^0-9]',value="", regex = True)


tabela_2.drop(columns =["Carimbo de data/hora","Endereço de e-mail","1 - Nome d(a) pesquisador(a)","2 - Número do processo (colocar no padrão CNJ com pontos e traço como está na planilha. EX: 1000347-33.2021.8.26.0114):"], inplace= True)

tabela_2.rename(columns={'3. O evento "prisão do agente" ocorreu:':'5.2. O evento "prisão do agente" ocorreu:',
                         '4. Caso haja informação sobre a prisão em outro processo, indique o(s) número(s) - se houver mais de um, separe por ponto e vírgula ";".':'5.3. Caso haja informação sobre a prisão em outro processo, indique o(s) número(s) - se houver mais de um, separe por ponto e vírgula ";".'},
                inplace = True)

df_final_aj.drop(columns = ['5.2. O evento "prisão do agente" ocorreu:','5.3. Caso haja informação sobre a prisão em outro processo, indique o(s) número(s) - se houver mais de um, separe por ponto e vírgula ";".'], inplace = True)

df_final_est.drop_duplicates(subset=["Carimbo de data/hora","1 - Nome do pesquisadora","2 - início do preenchimento","3 - Número do processo (se possível, colocar no padrão CNJ com pontos e traço. EX: 1000347-33.2021.8.26.0114):"], inplace= True, ignore_index=True)
df_final_aj_cm = pd.merge(df_final_aj, tabela_2, how = 'inner', on ='numero_limpo')

# print(df_final_aj_cm)

In [565]:
print(len(df_final_aj_cm))

159


In [566]:
print(len(df_final_aj))

4011


In [567]:
df_final_est = pd.concat([df_final_aj_cm,df_final_aj], ignore_index = True)
print(len(df_final_est))

4170


In [568]:
df_final_est.drop_duplicates(subset=["Carimbo de data/hora","1 - Nome do pesquisadora","2 - início do preenchimento","3 - Número do processo (se possível, colocar no padrão CNJ com pontos e traço. EX: 1000347-33.2021.8.26.0114):"], inplace= True, ignore_index=True)
print(len(df_final_est))

4011


In [569]:
list_num = ['50009375620188210058','0140359-45.2017.8.21.0001',"0401059-48.2017.8.07.0015","0000958-09.2019.8.15.0751",
            '0002262-74.2017.8.17.4011','0000324-70.2019.8.17.4012','000096617.2017.8.17.4011','1003334-41.2019.8.17.4001',
            '0700016-55.2017.8.18.0028','0100483-93.2020.8.20.0102','1000389-24.2020.8.23.0010','1000321-11.2019.8.23.0010',
            '1000426-85.2019.8.23.0010','1000252-13.2018.8.23.0010','1000207-09.2018.8.23.0010','1000165-57.2018.8.23.0010',
            '1000650-23.2019.8.23.0010','0004423-73.2017.8.25.0086']
list_vl = ["sim: morte externa","sim: morte externa","sim: morte interna","sim: morte interna",
           "sim: morte interna","sim: morte interna","sim: morte interna","sim: morte interna",
           "sim: morte interna","sim: morte interna","sim: morte interna","sim: morte interna",
           "sim: morte interna","sim: morte interna"]


for num,vl in zip(list_num,list_vl):
  idx = df_final_est[df_final_est ["3 - Número do processo (se possível, colocar no padrão CNJ com pontos e traço. EX: 1000347-33.2021.8.26.0114):"] == num].index.to_list()
  # print(idx)
  # print(df_final_est.loc[idx[0], "5 - É caso de interesse?"])
  df_final_est.loc[idx[0], "5 - É caso de interesse?"] = vl
  # print(df_final_est.loc[idx[0], "5 - É caso de interesse?"])



df_final_est.loc[3475, '35 - Tempo de pena - em caso de condenação (no processo analisado) - por favor coloque no padrão (X anos, y meses e z dias + Multa) e se houver mais de uma, separar por ponto e vírgula ";"'] = "NSA"


datetime_obj_f = datetime.datetime.strptime('28/03/2019', '%d/%m/%Y')
df_final_est.loc[3475,'40 -  Data da expedição do alvará de soltura da pessoa (colocar 010101 - para casos "sem informações" ** OU ** 020202 - para casos de "não se aplica")'] = datetime_obj_f

df_final_est.loc[3475,'74 - Documentos referentes à morte juntados aos autos (listar e separar por ponto e vírgula ( ";")'] = 'Certidão de óbito; Consulta de Preso da Secretaria de Segurança Pública do Rio Grande do Sul'

Exportando a planilha final

In [570]:
df_final_est.to_excel("df_final_est.xlsx", index = False)
files.download('df_final_est.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>